In [ ]:
import os
import sys
import time as t_util
import numpy as np
import pandas as pd
import yaml
import cftime
import xarray as xr

#My functions
sys.path.insert(0,'../functions/')
import functions_HeatWavesCities as fun_HWC


In [ ]:
#Read main paths
with open('../path_main.txt', 'r') as file:    path_main  = file.read()
with open('../path_EUR-11.txt', 'r') as file:  path_eur11 = file.read()
with open('../path_grids.txt', 'r') as file:   dir_grids  = file.read()
    
dir_CORDEX     = path_eur11
dir_scripts    = f'{path_main}Scripts/'
dir_names      = f'{path_main}Scripts/Model_lists/'
dir_orog       = f'{dir_CORDEX}historical/orog/'
dir_COR_out    = f'{path_main}Data/EURO-CORDEX/Variables/'
if not os.path.exists(dir_COR_out): os.mkdir(dir_COR_out)


## Prepare variables and parameters

In [ ]:
#Define cities
cities = ['Istanbul', 'Moscow', 'London', 'SaintPetersburg', 'Berlin', 'Madrid', 'Kyiv', 'Rome', 'Paris',
          'Bucharest', 'Minsk', 'Vienna', 'Hamburg', 'Warsaw', 'Budapest', 'Barcelona', 'Munich', 'Kharkiv',
          'Milan', 'Belgrade', 'Prague', 'NizhnyNovgorod', 'Kazan', 'Sofia', 'Brussels', 'Stockholm', 'Oslo',
          'Dublin', 'Lisbon', 'Vilnius', 'Copenhagen', 'Helsinki', 'Athens', 'Amsterdam', 'Riga', 'Zagreb']

#Define scenarios and variables
scenarios = ['historical', 'rcp85']
variables = ['tasmin', 'tasmax']

N = 3
N_str = str(N) + 'x' + str(N)

# Load city coordinates
fname_coords = dir_scripts + 'City_coordinates.yml'
with open(fname_coords, 'r') as file:
    city_coords = yaml.safe_load(file)

#Define models and RCPs which should be used
all_models = dict()
all_models['rcp26'] = []
all_models['rcp85'] = []
with open(dir_names + 'Models_CORDEX-EUR-11_RCP26.txt', 'r') as filehandle:
    for line in filehandle:
        all_models['rcp26'].append(eval(line[:-1]))
with open(dir_names + 'Models_CORDEX-EUR-11_RCP85.txt', 'r') as filehandle:
    for line in filehandle:
        all_models['rcp85'].append(eval(line[:-1]))
        
#Add models for historical
mod_85 = ["_".join(model) for model in all_models['rcp85']]
mod_26 = ["_".join(model) for model in all_models['rcp26']]
all_models['historical'] = [model.split('_') for model in sorted(list(set(mod_85).union(set(mod_26))))]


In [ ]:
all_models['rcp85'] = [ ['CNRM-CERFACS-CNRM-CM5', 'CLMcom-ETH-COSMO-crCLIM-v1-1', 'r1i1p1'],
                        ['CNRM-CERFACS-CNRM-CM5', 'ICTP-RegCM4-6', 'r1i1p1'],
                        ['CNRM-CERFACS-CNRM-CM5', 'MOHC-HadREM3-GA7-05', 'r1i1p1'],
                        ['MPI-M-MPI-ESM-LR', 'IPSL-WRF381P', 'r1i1p1'],
                        ['NCC-NorESM1-M', 'ICTP-RegCM4-6', 'r1i1p1']]

all_models['historical'] = [ ['CNRM-CERFACS-CNRM-CM5', 'CLMcom-ETH-COSMO-crCLIM-v1-1', 'r1i1p1'],
                        ['CNRM-CERFACS-CNRM-CM5', 'ICTP-RegCM4-6', 'r1i1p1'],
                        ['CNRM-CERFACS-CNRM-CM5', 'MOHC-HadREM3-GA7-05', 'r1i1p1'],
                        ['MPI-M-MPI-ESM-LR', 'IPSL-WRF381P', 'r1i1p1'],
                        ['NCC-NorESM1-M', 'ICTP-RegCM4-6', 'r1i1p1']]


## Prepare EURO-CORDEX data

In [ ]:
#Loop over scenarios
for scen in scenarios:

    models = all_models[scen]

    #Loop over models
    for model in models:
        
        #Select time limits
        if scen=='historical':
            time_sel = slice('1980', '2005')
        elif scen in ['rcp26', 'rcp85']:
            if (model[0]=='MOHC-HadGEM2-ES') and (model[1]=='MOHC-HadREM3-GA7-05'):
                time_sel = slice('2005', '2100')
            else:
                time_sel = slice('2006', '2100')

        print("_".join(model))
        
        #Initialize dict to store data
        data_coll = dict()
            
        #Loop over variables
        for variab in variables:

            print(" -" + variab, end='')

            #Get file names
            dir_files = dir_CORDEX + scen + '/' + variab + '/'
            fnames = [dir_files + file for file in os.listdir(dir_files) if model[0] in file and model[1] in file and model[2] in file]
            
            #Sort filenames
            fnames = sorted(fnames)

            #Delete December 2005 in RCP2.6 and RCP8.5 files for this model combination
            if (model[0]=='MOHC-HadGEM2-ES') and (model[1]=='ICTP-RegCM4-6') and scen in ['rcp26', 'rcp85']:
                fnames = [file for file in fnames if ('2005' not in file or '2006' not in file)]                

            #Merge single files to one large file
            t_sta = t_util.time()
            file_merge = dir_COR_out + 'CORDEX_merged_' + variab + '_' + scen + "_" + "_".join(model) + '_' + N_str + '_tmp.nc'
            if os.path.exists(file_merge): os.remove(file_merge)
            os.system('cdo mergetime ' + " ".join(fnames) + " " + file_merge)
            t_end = t_util.time()
            print(" -- " + "{:.1f}".format(t_end - t_sta), end='')
            
            #Open data set
            with xr.open_dataset(file_merge, use_cftime=True) as ds:
                data = ds.load()
                ds.close()

            #Convert °C to K
            if (model[0]=='CNRM-CERFACS-CNRM-CM5') and (model[1]=='CNRM-ALADIN53') and variab in ['tasmin', 'tasmax'] and scen in ['rcp26', 'rcp85']:
                attrs = data[variab].attrs
                data[variab] = data[variab] + 273.15
                data[variab].attrs = attrs
                
            # Convert from hPa to Pa
            if model[1]=='CNRM-ALADIN53' and variab=='ps':
                data[variab] = 100 * data[variab]
                
            #Correct wrong x- and y-values for CNRM-ALADIN53
            if model[1]=='CNRM-ALADIN53':
                data.x.values[107] = 1337.5
                data.y.values[107] = 1337.5
                
            #Convert longitude from [0, 360] to [-180, 180]
            if 'longitude' in data.coords:  lat_name, lon_name = 'latitude', 'longitude'
            elif 'lon' in data.coords:      lat_name, lon_name = 'lat', 'lon'
            if data[lon_name].max()>180:
                data[lon_name] = data[lon_name].where(data[lon_name]<180, ((data[lon_name] + 180) % 360) - 180)
                
            #Loop over cities
            for city in cities:

                #Get lat and lon of city
                lat_sel, lon_sel = city_coords[city]
                
                #Find grid point closest to city
                loc_city = (np.abs(data[lon_name] - lon_sel)) + (np.abs(data[lat_name] - lat_sel))
                ind_city = np.unravel_index(np.argmin(loc_city.values), loc_city.shape)
                
                #Select NxN box around grid point
                N1 = int(N/2 - 0.5)
                N2 = int(N/2 + 0.5)
                lat_rng  = slice(ind_city[0] - N1, ind_city[0] + N2)
                lon_rng  = slice(ind_city[1] - N1, ind_city[1] + N2)
                if 'rlat' in data.dims:   data_sel = data.isel(rlat=lat_rng, rlon=lon_rng)
                elif 'x' in data.dims:    data_sel = data.isel(y=lat_rng, x=lon_rng)
                else: sys.exit('Coordinate names could not be identified')

                #Save in dict
                data_coll[city + '_' + variab] = data_sel.load()
            
            #Remove temporarily merged file
            os.remove(file_merge)
            t_end = t_util.time()
            print(", " + "{:.1f}".format(t_end - t_sta))
            
        #Loop over cities
        for city in cities:
            
            #Define output folder
            dir_city = dir_COR_out + city  + '/'
            dir_save = dir_city + scen + '/'
            if not os.path.exists(dir_city): os.mkdir(dir_city)
            if not os.path.exists(dir_save): os.mkdir(dir_save)
            
            #Loop over variables
            for i2, variab in enumerate(variables):        

                #Add missing values in December 2099
                if (((model[0]=='MOHC-HadGEM2-ES') and (model[1]=='IPSL-WRF381P') and (scen=='rcp85')) or 
                    ((model[0]=='MOHC-HadGEM2-ES') and (model[1]=='MOHC-HadREM3-GA7-05') and (scen in ['rcp85', 'rcp26'])) or
                    ((model[0]=='MOHC-HadGEM2-ES') and (model[1]=='CLMcom-ETH-COSMO-crCLIM-v1-1') and (scen=='rcp85'))):
                    
                    fname_tmp = dir_COR_out + 'tmp-' + N_str + '_time.nc'
                    data_corr = data_coll[city + '_' + variab]
                    
                    if (model[0]=='MOHC-HadGEM2-ES') and (model[1]=='IPSL-WRF381P') and (scen=='rcp85'):
                        data_full = fun_HWC.add_missing_data(data_corr, fname_tmp, variab, '20991202', '20991231', 30)
                    elif (model[0]=='MOHC-HadGEM2-ES') and (model[1]=='MOHC-HadREM3-GA7-05') and (scen=='rcp26'):
                        data_full = fun_HWC.add_missing_data(data_corr, fname_tmp, variab, '20991230', '20991230', 1)
                    elif (model[0]=='MOHC-HadGEM2-ES') and (model[1]=='MOHC-HadREM3-GA7-05') and (scen=='rcp85'):
                        data_full = fun_HWC.add_missing_data(data_corr, fname_tmp, variab, '20991220', '20991230', 11)
                    elif (model[0]=='MOHC-HadGEM2-ES') and (model[1]=='CLMcom-ETH-COSMO-crCLIM-v1-1') and (scen=='rcp85'):
                        data_full = fun_HWC.add_missing_data(data_corr, fname_tmp, variab, '20990101', '20991230', 360)                    
                    
                    #Save corrected data in dict
                    data_full = data_full.sortby('time')
                    data_coll[city + '_' + variab] = data_full                   
                    
                #Convert to pandas dataframe
                data_variab = data_coll[city + '_' + variab].sel(time=time_sel)

                #Define lat and lon name
                if 'rlat' in data_variab.dims:   lat_name, lon_name = 'rlat', 'rlon'
                elif 'x' in data_variab.dims:    lat_name, lon_name = 'x', 'y'   
                
                #Put all variables in one dataframe
                if i2==0:
                    data_out = data_variab
                else:
                    data_variab['time'] = data_out.time
                    data_out[variab] = data_variab[variab]
                
                #Correct sea level pressure
                if variab=='psl':

                    #Read orography data and get temperature data
                    orog = fun_HWC.get_orog(model, city, city_coords[city], dir_orog)
                    Temp = data_out['tasmax']

                    #Correct pressure
                    T   = Temp.values
                    psl = data_variab['psl'].values
                    p_corr = fun_HWC.corr_press(psl, orog, T)

                    #Save in data frame and rename psl -> sp
                    data_out['psl'] = (('time', lat_name, lon_name), p_corr)
                    data_out = data_out.rename({'psl': 'sp'})
                    
            #Select data only from 1981 (because not all variables have data for 1980)
            if model[0]=='IPSL-IPSL-CM5A-MR' and model[1]=='KNMI-RACMO22E' and scen=='historical':
                data_out = data_out.sel(time=slice('1981', None))
                
            #Save data
            t1 = str(data_out.time[0].dt.year.values)
            t2 = str(data_out.time[-1].dt.year.values)
            fname_out = dir_save + "Variables-" + N_str + "_" + city + "_" + "_".join(model) + '_' + scen + '_day_' + t1 + "-" + t2 + ".nc"        
            if os.path.exists(fname_out): os.remove(fname_out)
            data_out.to_netcdf(fname_out)
